In [1]:
!pip install --user folium

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import pymongo
client = pymongo.MongoClient()
db = client.strava

In [94]:
search_string = "Derek B"

name_pipeline = [
    {"$match":  {"$text": {"$search": search_string }}},
    {"$limit": 10}, # without this limit, Jupyter has a hard time handling all the output
    {"$lookup": # find the associated user
         {
           "from": "users",
           "localField": "athlete_id",
           "foreignField": "user_id",
           "as": "user"
         }
    }
]

#import pprint
search_by_name = list(db.activity.aggregate(name_pipeline))
#pprint.pprint(search_by_location)

In [95]:
import folium
import random

m = folium.Map(location=[20,0],zoom_start=2)#location=search_coordinates, zoom_start=13)

# add the search area
"""
folium.CircleMarker(
    location=search_coordinates,
    radius=100,
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(m)
"""

popup_template="""
    <img src="{avatar}" width="100px" height="100px"></img><br>
    <center><b>{fullname}</b></center><br>
    <a href="{user_url}">Full profile</a>
    """

colors = ["red", "green", "blue", "black", "yellow"]
for result in search_by_name:
    fullname = result['user'][0]['full_name']
    avatar = result['user'][0]['avatar']
    user_url = result['user'][0]['user_url']
    if result['latlng'] is not None:
        tuples = [tuple(x) for x in result['latlng'] if x is not None]
    else:
        tuples = []
    
    # draw the run route
    folium.PolyLine(tuples, color=random.choice(colors), weight=2.5, opacity=0.5).add_to(m)
    
    # set up the user info popup
    popup_html = popup_template.format(fullname=fullname, avatar=avatar, user_url=user_url)
    
    # plot the start and end points of each run
    if len(tuples) != 0:
        folium.Marker(tuples[0], popup=popup_html, icon=folium.Icon(color='green')).add_to(m)
        folium.Marker(tuples[-1], popup=popup_html, icon=folium.Icon(color='red')).add_to(m)

m